In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import json
import csv
import webbrowser
from datetime import datetime
from urllib.parse import quote_plus

# Configuration
PHONE_NUMBER = "+2348148214845"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0'
}
NIGERIAN_SITES = [
    "nairaland.com",
    "jiji.ng",
    "jobberman.com.ng",
    "jumia.com.ng",
    "jamaa.com",
    "jamb.org.ng",
    "waecdirect.org",
    "cac.gov.ng"
]
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
JSON_OUTPUT = f"phone_intel_{PHONE_NUMBER}_{TIMESTAMP}.json"
CSV_OUTPUT = f"phone_intel_{PHONE_NUMBER}_{TIMESTAMP}.csv"

def format_phone_variations(phone):
    """Generate common phone number formatting variations"""
    base = phone.replace('+', '').replace(' ', '')
    variations = [
        phone,  # +2348148214845
        f"+{base}",  # +2348148214845
        base,  # 2348148214845
        base[3:],  # 8148214845 (without country code)
        f"{base[:4]} {base[4:7]} {base[7:10]} {base[10:]}",  # 2348 148 214 845
        f"{base[:4]}-{base[4:7]}-{base[7:10]}-{base[10:]}",  # 2348-148-214-845
        f"0{base[3:]}",  # 08148214845 (common Nigerian local format)
        f"{base[:4]} {base[4:7]} {base[7:]}",  # 2348 148 214845
        f"({base[:4]}) {base[4:7]}-{base[7:10]}-{base[10:]}",  # (2348) 148-214-845
        f"{base[:4]}.{base[4:7]}.{base[7:10]}.{base[10:]}",  # 2348.148.214.845
    ]
    return list(set(variations))  # Remove duplicates

def search_truecaller(phone):
    """Search Truecaller for phone number information"""
    print(f"[*] Checking Truecaller for {phone}...")
    url = f"https://www.truecaller.com/search/ng/{phone}"
    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract information
            name = soup.find('h1', class_='profile-name').text.strip() if soup.find('h1', class_='profile-name') else "Not Found"
            location = soup.find('div', class_='location').text.strip() if soup.find('div', class_='location') else "Not Found"
            carrier = soup.find('a', class_='carrier').text.strip() if soup.find('a', class_='carrier') else "Not Found"
            
            return {
                'source': 'Truecaller',
                'name': name,
                'location': location,
                'carrier': carrier,
                'url': url
            }
    except Exception as e:
        print(f"[!] Truecaller error: {str(e)}")
    return {'source': 'Truecaller', 'status': 'No results found'}

def search_social_media(platform, phone):
    """Search social media platforms for phone number"""
    print(f"[*] Checking {platform} for {phone}...")
    
    platforms = {
        'Facebook': 'https://www.facebook.com/public/',
        'Twitter': 'https://twitter.com/search?q=',
        'Instagram': 'https://www.instagram.com/accounts/login/?next=/',
        'LinkedIn': 'https://www.linkedin.com/search/results/all/?keywords='
    }
    
    try:
        url = f"{platforms[platform]}{quote_plus(phone)}"
        response = requests.get(url, headers=HEADERS, timeout=15)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            profiles = []
            
            # Platform-specific extraction
            if platform == 'Facebook':
                for result in soup.select('div._2pi9._2pi2'):
                    name = result.select_one('div._2b05').text.strip() if result.select_one('div._2b05') else "Unknown"
                    profile_url = result.select_one('a')['href'] if result.select_one('a') else "#"
                    image_url = result.select_one('img')['src'] if result.select_one('img') else ""
                    profiles.append({
                        'name': name,
                        'profile_url': f"https://facebook.com{profile_url}",
                        'image_url': image_url
                    })
            
            elif platform == 'Twitter':
                for result in soup.select('div.css-175oi2r.r-1wbh5a2.r-dnmrzs'):
                    name = result.select_one('span.css-1qaijid.r-bcqeeo.r-qvutc0.r-poiln3').text.strip() if result.select_one('span.css-1qaijid.r-bcqeeo.r-qvutc0.r-poiln3') else "Unknown"
                    handle = result.select_one('div.css-1dbjc4n.r-18u37iz').text.strip() if result.select_one('div.css-1dbjc4n.r-18u37iz') else "Unknown"
                    profile_url = result.select_one('a.css-175oi2r.r-1wbh5a2.r-dnmrzs.r-1ny4l3l')['href'] if result.select_one('a.css-175oi2r.r-1wbh5a2.r-dnmrzs.r-1ny4l3l') else "#"
                    profiles.append({
                        'name': name,
                        'handle': handle,
                        'profile_url': f"https://twitter.com{profile_url}"
                    })
            
            elif platform == 'Instagram':
                # Instagram requires login - we'll return search URL instead
                return {
                    'source': 'Instagram',
                    'message': 'Instagram requires login to search by phone number',
                    'search_url': url
                }
            
            elif platform == 'LinkedIn':
                for result in soup.select('li.reusable-search__result-container'):
                    name = result.select_one('span.entity-result__title-text').text.strip() if result.select_one('span.entity-result__title-text') else "Unknown"
                    headline = result.select_one('div.entity-result__primary-subtitle').text.strip() if result.select_one('div.entity-result__primary-subtitle') else ""
                    profile_url = result.select_one('a.app-aware-link')['href'] if result.select_one('a.app-aware-link') else "#"
                    profiles.append({
                        'name': name,
                        'headline': headline,
                        'profile_url': profile_url
                    })
            
            return {
                'source': platform,
                'results': profiles,
                'search_url': url
            }
    except Exception as e:
        print(f"[!] {platform} error: {str(e)}")
    
    return {'source': platform, 'status': 'No results found'}

def search_nigerian_sites(phone):
    """Search Nigerian sites through Google"""
    print(f"[*] Searching Nigerian sites for {phone}...")
    results = []
    base_url = "https://www.google.com/search?q="
    
    for site in NIGERIAN_SITES:
        try:
            time.sleep(random.uniform(2, 5))
            query = f"site:{site} {phone}"
            url = f"{base_url}{quote_plus(query)}"
            response = requests.get(url, headers=HEADERS, timeout=15)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                for result in soup.select('div.g'):
                    title = result.select_one('h3').text if result.select_one('h3') else "No Title"
                    link = result.select_one('a')['href'] if result.select_one('a') else "#"
                    desc = result.select_one('div.VwiC3b').text if result.select_one('div.VwiC3b') else "No Description"
                    
                    results.append({
                        'site': site,
                        'title': title,
                        'url': link,
                        'description': desc
                    })
        except Exception as e:
            print(f"[!] Error searching {site}: {str(e)}")
    
    return {'source': 'Nigerian Sites', 'results': results}

def search_google(phone_variations):
    """Search Google for references to the phone number"""
    print("[*] Searching Google...")
    results = []
    base_url = "https://www.google.com/search?q="
    
    for phone in phone_variations:
        try:
            time.sleep(random.uniform(2, 5))
            url = f"{base_url}{quote_plus(phone)}"
            response = requests.get(url, headers=HEADERS, timeout=15)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                for result in soup.select('div.g'):
                    title = result.select_one('h3').text if result.select_one('h3') else "No Title"
                    link = result.select_one('a')['href'] if result.select_one('a') else "#"
                    desc = result.select_one('div.VwiC3b').text if result.select_one('div.VwiC3b') else "No Description"
                    
                    results.append({
                        'query': phone,
                        'title': title,
                        'url': link,
                        'description': desc
                    })
        except Exception as e:
            print(f"[!] Google search error: {str(e)}")
    
    return {'source': 'Google', 'results': results}

def search_phone_validator(phone):
    """Check phone number validity and carrier information"""
    print(f"[*] Validating {phone}...")
    base_url = "https://www.phonevalidator.com/api/v2/phonesearch?apikey=demo&phone="
    
    try:
        clean_phone = ''.join(c for c in phone if c.isdigit() or c == '+')
        if clean_phone.startswith('+'):
            clean_phone = clean_phone[1:]
            
        url = f"{base_url}{clean_phone}"
        response = requests.get(url, headers=HEADERS, timeout=15)
        
        if response.status_code == 200:
            data = response.json()
            if data.get('success'):
                return {
                    'source': 'PhoneValidator',
                    'valid': data.get('valid', False),
                    'country': data.get('country', 'Unknown'),
                    'location': data.get('location', 'Unknown'),
                    'carrier': data.get('carrier', 'Unknown'),
                    'line_type': data.get('line_type', 'Unknown'),
                    'spam_score': data.get('spam_score', 0)
                }
    except Exception as e:
        print(f"[!] Phone validation error: {str(e)}")
    
    return {'source': 'PhoneValidator', 'status': 'No information available'}

def save_results(data):
    """Save results to JSON and CSV files"""
    # Save to JSON
    with open(JSON_OUTPUT, 'w') as json_file:
        json.dump(data, json_file, indent=2)
    
    # Save to CSV
    with open(CSV_OUTPUT, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Source', 'Type', 'Details'])
        
        for entry in data:
            source = entry['source']
            if 'results' in entry:
                results = entry['results']
                if isinstance(results, list):
                    for item in results:
                        if isinstance(item, dict):
                            details = "; ".join(f"{k}: {v}" for k, v in item.items())
                            writer.writerow([source, 'Result', details])
                        else:
                            writer.writerow([source, 'Result', item])
                elif isinstance(results, dict):
                    details = "; ".join(f"{k}: {v}" for k, v in results.items())
                    writer.writerow([source, 'Summary', details])
            elif 'status' in entry:
                writer.writerow([source, 'Status', entry['status']])
            else:
                details = "; ".join(f"{k}: {v}" for k, v in entry.items() if k != 'source')
                writer.writerow([source, 'Info', details])

def display_google_results(results):
    """Display Google results in a prominent format"""
    print("\n" + "="*60)
    print(f"GOOGLE RESULTS ({len(results)} FOUND)")
    print("="*60)
    
    for i, result in enumerate(results, 1):
        print(f"\nRESULT #{i}:")
        print(f"TITLE: {result.get('title', 'N/A')}")
        print(f"URL: {result.get('url', 'N/A')}")
        print(f"SNIPPET: {result.get('description', 'N/A')}")
        print("-"*60)

def display_social_results(platform, results):
    """Display social media results"""
    if not results or not isinstance(results, list):
        return
    
    print("\n" + "="*60)
    print(f"{platform.upper()} RESULTS ({len(results)} FOUND)")
    print("="*60)
    
    for i, profile in enumerate(results, 1):
        print(f"\nPROFILE #{i}:")
        for key, value in profile.items():
            print(f"{key.upper()}: {value}")
        print("-"*60)

def main():
    """Main investigation function"""
    phone_variations = format_phone_variations(PHONE_NUMBER)
    print(f"[+] Investigating phone number: {PHONE_NUMBER}")
    print(f"[+] Generated variations: {', '.join(phone_variations)}")
    
    # Run all searches
    all_results = [
        search_truecaller(PHONE_NUMBER),
        search_social_media('Facebook', PHONE_NUMBER),
        search_social_media('Twitter', PHONE_NUMBER),
        search_social_media('Instagram', PHONE_NUMBER),
        search_social_media('LinkedIn', PHONE_NUMBER),
        search_google(phone_variations),
        search_nigerian_sites(PHONE_NUMBER),
        search_phone_validator(PHONE_NUMBER)
    ]
    
    # Save results
    save_results(all_results)
    print(f"\n[+] Investigation complete!")
    print(f"[+] Results saved to {JSON_OUTPUT} and {CSV_OUTPUT}")
    
    # Display key findings
    print("\n=== KEY FINDINGS ===")
    
    # Extract and display Google results
    google_data = next((r for r in all_results if r['source'] == 'Google'), None)
    if google_data and 'results' in google_data:
        display_google_results(google_data['results'])
    
    # Extract and display social media results
    for platform in ['Facebook', 'Twitter', 'LinkedIn']:
        social_data = next((r for r in all_results if r['source'] == platform), None)
        if social_data and 'results' in social_data and social_data['results']:
            display_social_results(platform, social_data['results'])
    
    # Extract Nigerian sites results
    nigerian_data = next((r for r in all_results if r['source'] == 'Nigerian Sites'), None)
    if nigerian_data and 'results' in nigerian_data and nigerian_data['results']:
        print("\n" + "="*60)
        print("NIGERIAN SITES RESULTS")
        print("="*60)
        for result in nigerian_data['results']:
            print(f"\nSITE: {result['site']}")
            print(f"TITLE: {result['title']}")
            print(f"URL: {result['url']}")
            print("-"*60)
    
    # Open results in browser for quick review
    if google_data and 'results' in google_data:
        print("\n[+] Opening Google results in browser...")
        for i, result in enumerate(google_data['results'][:3], 1):
            print(f"Opening result {i}: {result['url']}")
            webbrowser.open(result['url'])
            time.sleep(2)

if __name__ == "__main__":
    main()

[+] Investigating phone number: +2348148214845
[+] Generated variations: 2348148214845, 2348.148.214.845, (2348) 148-214-845, 2348-148-214-845, 08148214845, 2348 148 214 845, 8148214845, +2348148214845, 2348 148 214845
[*] Checking Truecaller for +2348148214845...
[*] Checking Facebook for +2348148214845...
[*] Checking Twitter for +2348148214845...
[*] Checking Instagram for +2348148214845...
[*] Checking LinkedIn for +2348148214845...
[*] Searching Google...
[*] Searching Nigerian sites for +2348148214845...
[*] Validating +2348148214845...

[+] Investigation complete!
[+] Results saved to phone_intel_+2348148214845_20250616_154737.json and phone_intel_+2348148214845_20250616_154737.csv

=== KEY FINDINGS ===

GOOGLE RESULTS (0 FOUND)

[+] Opening Google results in browser...


In [3]:
def display_csv_simple(csv_file):
    """Basic CSV display without external packages"""
    print(f"\nCSV Contents of {csv_file}:")
    print("-" * 100)
    
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        
        # Print headers
        headers = next(reader)
        print(" | ".join(headers))
        print("-" * 100)
        
        # Print rows
        for i, row in enumerate(reader, 1):
            print(f"{i}. {' | '.join(row)}")
    
    print("-" * 100)

# Call in main():
display_csv_simple(CSV_OUTPUT)


CSV Contents of phone_intel_+2348148214845_20250616_154737.csv:
----------------------------------------------------------------------------------------------------
Source | Type | Details
----------------------------------------------------------------------------------------------------
1. Truecaller | Info | name: Not Found; location: Not Found; carrier: Not Found; url: https://www.truecaller.com/search/ng/+2348148214845
2. Facebook | Status | No results found
3. Instagram | Info | message: Instagram requires login to search by phone number; search_url: https://www.instagram.com/accounts/login/?next=/%2B2348148214845
4. PhoneValidator | Status | No information available
----------------------------------------------------------------------------------------------------


In [4]:
# Add this to the end of main()
def process_csv_data(csv_file):
    """Process CSV data programmatically"""
    print("\nProcessing CSV data...")
    
    data_by_source = {}
    
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        
        for row in reader:
            source = row['Source']
            if source not in data_by_source:
                data_by_source[source] = []
            data_by_source[source].append(row)
    
    # Example: Show Google results
    if 'Google' in data_by_source:
        print("\nGoogle Results:")
        for result in data_by_source['Google']:
            print(f" - {result['Details']}")
    
    # Example: Show Truecaller results
    if 'Truecaller' in data_by_source:
        print("\nTruecaller Results:")
        for result in data_by_source['Truecaller']:
            print(f" - {result['Details']}")
    
    return data_by_source

# Call in main()
csv_data = process_csv_data(CSV_OUTPUT)


Processing CSV data...

Truecaller Results:
 - name: Not Found; location: Not Found; carrier: Not Found; url: https://www.truecaller.com/search/ng/+2348148214845


In [7]:
import sqlite3

conn = sqlite3.connect('customer_db.db')
cursor = conn.cursor()

# Create table
cursor.execute('''CREATE TABLE IF NOT EXISTS customers 
                  (id INTEGER PRIMARY KEY, name TEXT, email TEXT)''')

# Insert data
cursor.execute("INSERT INTO customers (name, email) VALUES (?, ?)", 
               ('Jane Doe', 'jane@example.com'))
conn.commit()

# Query data
cursor.execute("SELECT * FROM customers")
print(cursor.fetchall())
conn.close()

[(1, 'Jane Doe', 'jane@example.com')]
